In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
learned_data_path = "../clean_learned_EEG/combined_learned.csv"
not_learned_data_path = "../clean_not_learned_EEG/combined_not_learned.csv"

learned_data = pd.read_csv(learned_data_path, header = None)
not_learned_data = pd.read_csv(not_learned_data_path, header = None)

In [5]:
# checking how much data for each category
print(f"original learned shape: {learned_data.shape}")
print(f"original not_learned shape: {not_learned_data.shape}")

# make them equal to remove bias
learned_data = learned_data.sample(frac=1, random_state=42).reset_index(drop=True)
not_learned_data = not_learned_data.sample(frac=1, random_state=42).reset_index(drop=True)

# get length 
learned_length = learned_data.shape[0]
not_learned_length = not_learned_data.shape[0]

new_learned_data = learned_data.drop(index=range(not_learned_length, learned_length))
new_not_learned_data = not_learned_data.drop(index=range(learned_length, not_learned_length))

# print results
print(f"new learned shape: {new_learned_data.shape}")
print(f"new not_learned shape: {not_learned_data.shape}")

original learned shape: (746, 2456)
original not_learned shape: (141, 2456)
new learned shape: (141, 2456)
new not_learned shape: (141, 2456)


In [6]:
#Label Data

new_learned_data['label'] = 1
new_not_learned_data['label'] = 0

In [7]:
#Combine and Shuffle

full_data = pd.concat([new_learned_data, new_not_learned_data], ignore_index = True)
full_data = full_data.sample(frac = 1).reset_index(drop = True)

In [8]:
#Split between X and Y Data
#X is 42 rows (epochs) 21 Learned, 21 Not Learned
#Y is list with length 42 of 0 (NL) and 1 (L)

X = full_data.drop('label', axis = 1).values
Y = full_data['label'].values

In [9]:
#Standardize Features

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
#Split Data into Train and Test

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [26]:
#Building Neural Network Model

model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),                     # Prevent overfitting

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),                     # Prevent overfitting

    Dense(1, activation='sigmoid')   # Binary output
])

# Compile with optimizer and loss function
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Add early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,               # Stop after 10 epochs of no improvement
    restore_best_weights=True # Keep the best model
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=16,
    validation_split=0.1,
    callbacks=[early_stop],     # Use the early stopping
    verbose=1
)

y_pred = model.predict(X_test)
y_pred_labels = (y_pred > 0.5).astype("int32")  # convert probabilities to 0 or 1

# Print accuracy
print("Accuracy:", accuracy_score(y_test, y_pred_labels))

Epoch 1/1000


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5861 - loss: 0.9622 - val_accuracy: 0.5652 - val_loss: 0.7817
Epoch 2/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6721 - loss: 0.6949 - val_accuracy: 0.5652 - val_loss: 0.8225
Epoch 3/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6572 - loss: 0.7013 - val_accuracy: 0.5652 - val_loss: 0.8482
Epoch 4/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6883 - loss: 0.6241 - val_accuracy: 0.5652 - val_loss: 0.7467
Epoch 5/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7227 - loss: 0.5489 - val_accuracy: 0.6087 - val_loss: 0.7480
Epoch 6/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7001 - loss: 0.6198 - val_accuracy: 0.6087 - val_loss: 0.7244
Epoch 7/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7275 - loss: 0.5601 - val_accuracy: 0.6522 - val_loss: 0.7135
Epoch 8/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7076 - loss: 0.6323 - val_accuracy: 0.6087 - val_lo